In [3]:
from langchain_community.document_loaders import TextLoader # takes raw text of descriptions and converts for langchain to understand
from langchain_text_splitters import CharacterTextSplitter # for splitting documents
from langchain_openai import OpenAIEmbeddings # converting chunks into embeddings
from langchain_chroma import Chroma # for storing all embeddings into a vector database

In [ ]:
# from langchain.embeddings import HuggingFaceEmbeddings
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# db_books = Chroma.from_documents(
#     documents,
#     embedding=embedding_model
# )

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
import pandas as pd
movies = pd.read_csv('movies_cleaned.csv')

In [11]:
movies

,id,name,date,tagline,description,minute,rating,theme,genre,tagged_description
0,1000001,Barbie,2023.0,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114.0,3.86,"Humanity and the world around us, Crude humor ...","Comedy, Adventure",1000001 Barbie and Ken are having the time of ...
1,1000002,Parasite,2019.0,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56,"Humanity and the world around us, Intense viol...","Comedy, Thriller, Drama","1000002 All unemployed, Ki-taek's family takes..."
2,1000003,Everything Everywhere All at Once,2022.0,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140.0,4.30,"Humanity and the world around us, Moving relat...","Science Fiction, Adventure, Comedy, Action",1000003 An aging Chinese immigrant is swept up...
3,1000004,Fight Club,1999.0,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27,"Intense violence and sexual transgression, Hum...",Drama,1000004 A ticking-time-bomb insomniac and a sl...
4,1000005,La La Land,2016.0,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09,"Song and dance, Humanity and the world around ...","Drama, Comedy, Music, Romance","1000005 Mia, an aspiring actress, serves latte..."
...,...,...,...,...,...,...,...,...,...,...
24414,1664306,Fixerr,2019.0,NaN,ATS Officer Jayveer Malik’s stands suspended a...,324.0,NaN,"Thrillers and murder mysteries, High speed and...",Drama,1664306 ATS Officer Jayveer Malik’s stands sus...
24415,1693420,"LSD - Love, Scandal and Doctors",2021.0,NaN,"Love, Scandal and Doctors is a story of 5 medi...",300.0,NaN,"Thrillers and murder mysteries, High speed and...","Drama, Mystery","1693420 Love, Scandal and Doctors is a story o..."
24416,1702871,Olessia,2019.0,NaN,Film with a soundtrack featuring G. Venugopal ...,NaN,NaN,"Song and dance, Dazzling vocal performances an...",Drama,1702871 Film with a soundtrack featuring G. Ve...
24417,1762425,Pretty Things,2005.0,NaN,A look into the world of 20th century burlesqu...,90.0,NaN,"Song and dance, Erotic relationships and desire",Documentary,1762425 A look into the world of 20th century ...


### First going to want to convert pandas dataframe into a text file, for LangChain readability purposes

In [19]:
movies["tagged_description"].to_csv('tagged_description.txt', sep='\n', index=False, header=False)

In [22]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
# chunk_size = 0 to ensure that the chunks are separated based on the separator
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator='\n')
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 237, which is longer than the specified 0
Created a chunk of size 172, which is longer than the specified 0
Created a chunk of size 187, which is longer than the specified 0
Created a chunk of size 307, which is longer than the specified 0
Created a chunk of size 370, which is longer than the specified 0
Created a chunk of size 108, which is longer than the specified 0
Created a chunk of size 206, which is longer than the specified 0
Created a chunk of size 177, which is longer than the specified 0
Created a chunk of size 451, which is longer than the specified 0
Created a chunk of size 247, which is longer than the specified 0
Created a chunk of size 470, which is longer than the specified 0
Created a chunk of size 138, which is longer than the specified 0
Created a chunk of size 170, which is longer than the specified 0
Created a chunk of size 361, which is longer than the specified 0
Created a chunk of size 377, which is longer than the specified 0
Created a 

In [23]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='1000001 Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.')

### Now to stored embeddings into a vector database

In [25]:
pip install -U langchain-huggingface

  Obtaining dependency information for langchain-huggingface from https://files.pythonhosted.org/packages/9d/f8/77a303ddc492f6eed8bf0979f2bc6db4fa6eb1089c5e9f0f977dd87bc9c2/langchain_huggingface-0.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for sentence-transformers>=2.6.0 from https://files.pythonhosted.org/packages/32/58/770e1e762893abbfe3cd048f1ed1ea6e00122a195651ea98fb27f55ad17a/sentence_transformers-4.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.11.0 from https://files.pythonhosted.org/packages/18/cf/ae99bd066571656185be0d88ee70abc58467b76f2f7c8bfeb48735a71fe6/torch-2.6.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/62/27/585859e72e117fe861c2079bcba35591a84f801e21bc1ab85bce6ce60305/scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/f5/6f/e6e5aff77ea2a


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_books = Chroma.from_documents(
    documents,
    embedding=embedding_model
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Owner\PycharmProjects\movie-recommender\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owner\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
query = "A movie about aliens"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='93220b40-a44d-4c68-a5a0-74fa14523393', metadata={'source': 'tagged_description.txt'}, page_content='"1079381 A movie created from ""found footage"" of a lost military expedition that exposes the origins of life on earth."'),
 Document(id='85933fc9-1706-4a26-8622-05d2857e3ed4', metadata={'source': 'tagged_description.txt'}, page_content='"1059610 A farce satirizing extraterrestrial horror movies such as Alien.  Alternate titles include ""The Creature Wasn\'t Nice"" and ""Naked Space"" (1999 re-release trying to capitalize on Nielsen\'s ""Naked Gun"" success)."'),
 Document(id='1daf92d9-b810-46b6-80e0-cae467ffa985', metadata={'source': 'tagged_description.txt'}, page_content='1027934 A group of cave explorers are confronted in an underground cavern by a mysterious living rock. Little do they know that it bears home to deadly, flesh-eating creatures, intent on wiping out the entire human population. An ultra low-budget, unofficial and unauthorized sequel to the 1979 film Ali

In [33]:
movies[movies["id"] == int(docs[0].page_content.split()[0].replace("\"", "").strip())]

,id,name,date,tagline,description,minute,rating,theme,genre,tagged_description
23068,1079381,Alien Origin,2012.0,The Footage is Real. The Proof is Irrefutable.,"A movie created from ""found footage"" of a lost...",90.0,2.45,"Monsters, aliens, sci-fi and the apocalypse, A...","Science Fiction, Thriller, Fantasy","1079381 A movie created from ""found footage"" o..."


In [34]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    movies_list = []

    for i in range(0, len(recs)):
        movies_list += [int(recs[i].page_content.strip('"').split()[0])]

    return movies[movies["id"].isin(movies_list)]

In [38]:
retrieve_semantic_recommendations("A movie about aliens")

,id,name,date,tagline,description,minute,rating,theme,genre,tagged_description
494,1000496,Aliens,1986.0,This Time It's War,When Ripley's lifepod is found by a salvage cr...,137.0,4.15,"Monsters, aliens, sci-fi and the apocalypse, H...","Science Fiction, Horror, Thriller, Action",1000496 When Ripley's lifepod is found by a sa...
1491,1001501,Mars Attacks!,1996.0,Nice planet. We'll take it!,'We come in peace' is not what those green men...,106.0,3.22,"Crude humor and satire, Monsters, aliens, sci-...","Science Fiction, Fantasy, Comedy",1001501 'We come in peace' is not what those g...
2041,1002067,Killer Klowns from Outer Space,1988.0,It’s Craazzy!,Aliens who look like clowns come from outer sp...,88.0,3.31,"Crude humor and satire, Horror, the undead and...","Science Fiction, Horror, Comedy",1002067 Aliens who look like clowns come from ...
2157,1002184,Galaxy Quest,1999.0,A comedy of Galactic Proportions.,The stars of a 1980s sci-fi show—now eking out...,101.0,3.70,"Crude humor and satire, Monsters, aliens, sci-...","Comedy, Science Fiction, Adventure",1002184 The stars of a 1980s sci-fi show—now e...
3759,1003858,Aliens vs Predator: Requiem,2007.0,The Last Place On Earth We Want To Be Is In Th...,The iconic creatures from two of the scariest ...,94.0,1.61,"Monsters, aliens, sci-fi and the apocalypse, H...","Science Fiction, Fantasy, Action, Horror, Thri...",1003858 The iconic creatures from two of the s...
3915,1004026,Starman,1984.0,"In 1977 Voyager II was launched into space, in...",When an alien takes the form of a young widow'...,115.0,3.60,"Monsters, aliens, sci-fi and the apocalypse, I...","Fantasy, Romance, Science Fiction, Drama",1004026 When an alien takes the form of a youn...
4365,1004507,Evolution,2001.0,Coming to wipe that silly smile off your planet.,A comedy that follows the chaos that ensues wh...,101.0,2.81,"Crude humor and satire, Monsters, aliens, sci-...","Action, Comedy, Science Fiction",1004507 A comedy that follows the chaos that e...
4465,1004613,Monsters,2010.0,"After Six Years, They're No Longer Aliens. The...",Six years ago NASA discovered the possibility ...,94.0,3.21,"Monsters, aliens, sci-fi and the apocalypse, S...","Drama, Science Fiction, Thriller",1004613 Six years ago NASA discovered the poss...
5026,1005234,Coneheads,1993.0,Young ones! Parental units! We summon you!,A pair of aliens arrive on Earth to prepare fo...,87.0,2.59,"Crude humor and satire, Monsters, aliens, sci-...","Science Fiction, Comedy, Family",1005234 A pair of aliens arrive on Earth to pr...
5646,1005947,Earth Girls Are Easy,1988.0,Valerie's about to discover how far a girl has...,"In this musical comedy, Valerie is dealing wit...",100.0,3.14,"Crude humor and satire, Song and dance, Relati...","Science Fiction, Comedy","1005947 In this musical comedy, Valerie is dea..."
